In [35]:
import numpy as np
import pandas as pd
import datetime
import time
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.model_selection import train_test_split

## 依照參數回傳每人的資料 
輸入格式(資料名(combined), 年, 季(1 -> 1~3月 ...))

In [36]:
def get_season(combined, y, s):
    # if s == 4:
    #     lower = datetime.datetime(y,s*3-2,1,0,0)
    #     upper = datetime.datetime(y+1,1,1,0,0)
    # elif s < 4 and s > 0:
    #     lower = datetime.datetime(y,s*3-2,1,0,0)
    #     upper = datetime.datetime(y,s*3+1,1,0,0)
    # else:
    #     print('s must be 1, 2, 3 or 4')
    #     return 0
    if s == 4:
        lower = str(y)+'-10-01'
        upper = str(y+1)+'-01-01'
    elif s == 3:
        lower = str(y)+'-07-01'
        upper = str(y)+'-10-01'
    elif s == 1:
        lower = str(y)+'-01-01'
        upper = str(y)+'-04-01'
    elif s == 2:
        lower = str(y)+'-04-01'
        upper = str(y)+'-07-01'
    else:
        print('s must be 1, 2, 3 or 4')
        return 0
    print(lower, upper)
    
    ids = combined['UnifiedUserId'].unique()
    combined = combined[(combined['OrderDateTime'] < upper) & (combined['OrderDateTime'] > lower)]
  
    lst = []
    for i in ids:
        ele = {'UnifiedUserId': i,
               'MemberCardLevel': 0,
               'Gender': 'Ungiven',
               'Age': 0,
               'order': 0, #本季主單數
               'orderSlave': 0, #本季子單數
               'low': 0, #本季低價位商品購買數
               'normal': 0, #本季中價位商品購買數
               'high': 0, #本季高價位商品購買數
               'sales': 0,# 本季總消費
               'SalePageId':""
                }
        lst.append(ele)
        
    re = pd.DataFrame(lst).set_index('UnifiedUserId').copy()
    
    for index, r in combined.iterrows():
        idx = r['UnifiedUserId']
        re.loc.__setitem__((idx, ('Gender')), r['Gender'])
        re.loc.__setitem__((idx, ('MemberCardLevel')), r['MemberCardLevel'])
        re.loc.__setitem__((idx, ('Age')), int((datetime.datetime.today() - datetime.datetime.strptime(r['Birthday'], '%Y-%m-%d')).days/365))
        re.loc.__setitem__((idx, ('orderSlave')), r['TsCount'] + re.loc[idx]['orderSlave'])
        re.loc.__setitem__((idx, ('order')), 1 + re.loc[idx]['order'])
        re.loc.__setitem__((idx, ('sales')), r['TotalSalesAmount'] + re.loc[idx]['sales'])
        re.loc.__setitem__((idx, ('high')), r['high'] + re.loc[idx]['high'])
        re.loc.__setitem__((idx, ('normal')), r['normal'] + re.loc[idx]['normal'])
        re.loc.__setitem__((idx, ('low')), r['low'] + re.loc[idx]['low'])
        re.loc.__setitem__((idx, ('SalePageId')), r['SalePageId'] + "/" + re.loc[idx]['SalePageId'])
        
        

    re['avgSales'] = re['sales']/re['order']
                
                
    return re

## 讀取資料

In [37]:
member = pd.read_csv('91APP_MemberData.csv')
order = pd.read_csv('91APP_OrderData.csv')
product = pd.read_excel("user_tag.xlsx").dropna()

C:\Users\timmy\AppData\Local\Temp\ipykernel_28212\1633401050.py:2: DtypeWarning: Columns (5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  order = pd.read_csv('91APP_OrderData.csv')


In [38]:
member = member[['UnifiedUserId', 'Gender', 'Birthday', 'MemberCardLevel']]
member.head(3)

,UnifiedUserId,Gender,Birthday,MemberCardLevel
0,wDnLYnQDE5Nt/TBttarrEw==,Female,1987-11-12,10
1,PfTmZ2HBNiyYJrv0kLZpnw==,Female,1990-09-02,10
2,MajPkebcNSUBGwNtJOjVMA==,Female,1985-09-08,10


In [39]:
order = order[['TradesGroupCode', 'OrderDateTime', 'TsCount', 'Qty', 'TotalSalesAmount']]
order.head(3)

,TradesGroupCode,OrderDateTime,TsCount,Qty,TotalSalesAmount
0,5uPmVe+VOTmLhDIRYqZ0X9WEZkAr2lq1GaDvxM2C/fw=,2016-04-29 09:56:00,1,-1,-100.0
1,5uPmVe+VOTmLhDIRYqZ0X9WEZkAr2lq1GaDvxM2C/fw=,2016-04-29 09:56:00,2,2,1284.0
2,VTI/4oAuHAlVolhrBpXzm/k7eAzdN4fsM2NjCE6YLEI=,2016-04-29 11:04:00,1,1,1184.0


In [40]:
df = pd.read_csv('./data.csv')[['UnifiedUserId', 'TradesGroupCode', 'low', 'normal', 'high']]

## 在每個主單中加入 1)高價位單品數 2)中價位單品數 3)低價位的單品數

所有子單單價的前25%為高價位，中間50%為中價位，後25%為低價位 (不含0圓商品)

In [41]:
combine = pd.merge(order, df, left_on = 'TradesGroupCode', right_on = 'TradesGroupCode', how = 'left')

In [42]:
combine.head(3)

,TradesGroupCode,OrderDateTime,TsCount,Qty,TotalSalesAmount,UnifiedUserId,low,normal,high
0,5uPmVe+VOTmLhDIRYqZ0X9WEZkAr2lq1GaDvxM2C/fw=,2016-04-29 09:56:00,1,-1,-100.0,xjOctGt+CvE/+soyAS+LVA==,1.0,1.0,0.0
1,5uPmVe+VOTmLhDIRYqZ0X9WEZkAr2lq1GaDvxM2C/fw=,2016-04-29 09:56:00,2,2,1284.0,xjOctGt+CvE/+soyAS+LVA==,1.0,1.0,0.0
2,VTI/4oAuHAlVolhrBpXzm/k7eAzdN4fsM2NjCE6YLEI=,2016-04-29 11:04:00,1,1,1184.0,Agoqyv2QhggU0nmPX6tVJA==,0.0,1.0,0.0


In [43]:
combined = pd.merge(combine, member, left_on = 'UnifiedUserId', right_on = 'UnifiedUserId', how = 'left')

In [44]:
combined = combined[combined['Qty'] >= 0]

In [45]:
combined.head()

,TradesGroupCode,OrderDateTime,TsCount,Qty,TotalSalesAmount,UnifiedUserId,low,normal,high,Gender,Birthday,MemberCardLevel
1,5uPmVe+VOTmLhDIRYqZ0X9WEZkAr2lq1GaDvxM2C/fw=,2016-04-29 09:56:00,2,2,1284.0,xjOctGt+CvE/+soyAS+LVA==,1.0,1.0,0.0,NaN,1900-01-01,10.0
2,VTI/4oAuHAlVolhrBpXzm/k7eAzdN4fsM2NjCE6YLEI=,2016-04-29 11:04:00,1,1,1184.0,Agoqyv2QhggU0nmPX6tVJA==,0.0,1.0,0.0,NaN,0001-01-01,10.0
3,BmdhnTN8wcHjLbMp8O3awDOib/WiGtv9yazSs3H5Jnk=,2016-04-29 11:05:00,1,1,1184.0,xvZqF1Tlc8cGPJQwyRT24A==,1.0,0.0,0.0,NaN,1900-01-01,10.0
4,AlDaYlS/Oda/FRWZqGNt1kV4N6UhQ0tNbA2y7lmS5pI=,2016-05-02 10:29:00,1,1,1080.0,R+nvCLvUXeVCC4OYsSjSzA==,0.0,1.0,0.0,Female,1984-06-05,10.0
5,LZqYjv3x9IzUf5jNUtFz/3vijWR+KcTn3RzfTQF2yXs=,2016-05-03 10:29:00,2,2,1774.0,ftOkl2eeDxYeBcSZX3UiTA==,1.0,1.0,0.0,Female,0076-09-25,10.0


In [46]:
l = []
for i in combined['OrderDateTime']:
    l.append(i.split()[0])
combined['OrderDateTime'] = l

In [47]:
ProductTag = list(product["SalePageId"].unique())
print(ProductTag)

['小白鞋', '菱格紋包', '鞋用配件', '短靴', '拖鞋', '贈品', '跟鞋', '婚鞋', '膝靴', 'NG鞋', '身體保養', '鞋墊', '老爹鞋', '涼拖鞋', '穆勒鞋', '牛津鞋', '日用品', '厚底鞋', '尖頭鞋', '帆布鞋', '涼鞋', '長靴', '襪靴', '便鞋', '平底鞋', '小包', '皮夾x卡夾x零錢包', '紳士鞋', '服裝衣物', '懶人鞋', '休閒鞋', '包鞋', '飾品配件', '黛妃包', '包用配件', '凱莉包', '高跟鞋', '草編鞋', '踝靴', '雪靴', '水桶包', '餅乾鞋', '軍靴', '提袋', '尖頭靴', '馬丁靴', '肩背包', '娃娃鞋', '腰包', '低跟鞋', '多功能包', '方包', '雨靴', '方頭鞋', '後背包', '休閒包', '莫卡辛鞋', '劍橋包', '鞋拔', '相機包', 'NG包', '編織包', '球鞋', '腋下包', '樂福鞋', '柏金包', '童鞋', '托特包', '流蘇包', '摺疊鞋', '藤編包', '馬鞍包', '機車靴', '機車包', '工程靴', '慢跑鞋', '帆船鞋', '鍊條包', '拓特包', '短鞋', '跟靴', '蛋糕鞋', '豆豆鞋', '艾瑪包', '情侶鞋', '運動鞋', '瑪麗珍鞋']


In [48]:

combinedWithProduct = product.merge( combined, on = 'TradesGroupCode', how = 'left')

In [49]:
combinedWithProduct.describe()


,Unnamed: 0,TsCount,Qty,TotalSalesAmount,low,normal,high,MemberCardLevel
count,7.310830e+05,731083.000000,731083.000000,731083.000000,731083.000000,731083.000000,731083.000000,731083.000000
mean,1.062197e+06,2.920901,3.066633,2419.015292,1.587338,1.068906,0.358080,14.084912
std,2.784831e+05,2.221187,2.610242,1578.888564,1.860340,0.975659,0.691588,7.352791
min,2.000000e+00,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8.295345e+05,1.000000,1.000000,1480.000000,0.000000,0.000000,0.000000,10.000000
50%,1.065257e+06,2.000000,2.000000,1943.000000,1.000000,1.000000,0.000000,10.000000
75%,1.294772e+06,4.000000,4.000000,3000.000000,2.000000,1.000000,1.000000,20.000000
max,1.542842e+06,33.000000,61.000000,50519.000000,26.000000,17.000000,12.000000,40.000000


In [50]:
def AgeCoding(df):
    df.loc[df.Age < 21,'Age']=1
    df.loc[(df.Age > 20) & (df.Age < 31),'Age'] = 2
    df.loc[(df.Age > 30) & (df.Age < 41),'Age'] = 3
    df.loc[(df.Age > 40) & (df.Age < 51),'Age'] = 4
    df.loc[(df.Age > 50) & (df.Age < 61),'Age'] = 5
    df.loc[df.Age > 60,'Age'] = 6
    return df

def GetTag(df, TagList): ##幫每個Tag做Coding
    Dic = {}
    for i in TagList:    
        CodingList = []
        for j in df["SalePageId"]:
            words = j.split("/")
            count = 0
            for k in words:
                if k == i:
                    CodingList.append(1)
                    count += 1
                    break
            if count == 0:
                CodingList.append(0)
        Dic[i] = CodingList
    return Dic

def ModelTraining(df, tags): ##對每個Tag做Training
    score = 0
    for i in tags.keys():
        print("這是 " + i + " 的分類器")
        df_c = df.copy()
        df_c[i] = tags[i]
        
        ListFilyer = ["ReviseMemberCardLevel", "Age", "order", "orderSlave", "low", "normal", "high", "SalesLog"]
        ListFilyer.append(i)
        
        train_data, test_data = train_test_split(df_c[ListFilyer], random_state=777, train_size=0.9)
        ListFilyer.pop()
        X_train = train_data[ListFilyer]
        Y_train = train_data[i]
        X_test = test_data[ListFilyer]
        Y_test = test_data[i]
        
        RF_model = RandomForestClassifier(n_estimators = 100)
        RF_model = RF_model.fit(X_train, Y_train)
        train_score = RF_model.score(X_train, Y_train)
        print("Training score :", train_score)

        Y_predict = RF_model.predict(X_test)
        print('準確率 :', accuracy_score(Y_test, Y_predict))

        conf_mat = confusion_matrix(Y_test, Y_predict)
        print(conf_mat)
        if train_score == 1:
            score += 1
    return score

    

### Season 1 (1-3月)(2014-2020)

In [51]:
result = get_season(combinedWithProduct, 2014, 1)
for i in range(2015, 2021,1):
    a = get_season(combinedWithProduct, i, 1)
    result = pd.concat([result, a])
ResultSeason1 = result.dropna()

2014-01-01 2014-04-01
2015-01-01 2015-04-01
2016-01-01 2016-04-01
2017-01-01 2017-04-01
2018-01-01 2018-04-01
2019-01-01 2019-04-01
2020-01-01 2020-04-01


In [52]:
ResultSeason1 = AgeCoding(ResultSeason1)
ResultSeason1["SalesLog"] = np.log(ResultSeason1["sales"])
ResultSeason1["ReviseMemberCardLevel"] = ResultSeason1["MemberCardLevel"] / 10

C:\Users\timmy\AppData\Local\Temp\ipykernel_28212\1909290146.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ResultSeason1["SalesLog"] = np.log(ResultSeason1["sales"])
C:\Users\timmy\AppData\Local\Temp\ipykernel_28212\1909290146.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ResultSeason1["ReviseMemberCardLevel"] = ResultSeason1["MemberCardLevel"] / 10


In [53]:
TagDictionary = GetTag(ResultSeason1, ProductTag)

In [54]:
NumberOfScore1 = ModelTraining(ResultSeason1, TagDictionary)

這是 小白鞋 的分類器
Training score : 0.9033115157799235
準確率 : 0.8137026239067056
[[2565  198]
 [ 441  226]]
這是 菱格紋包 的分類器
Training score : 1.0
準確率 : 1.0
[[3430]]
這是 鞋用配件 的分類器
Training score : 0.9875899164020479
準確率 : 0.9014577259475218
[[2834  147]
 [ 191  258]]
這是 短靴 的分類器
Training score : 0.9468278141403668
準確率 : 0.8594752186588921
[[2795  163]
 [ 319  153]]
這是 拖鞋 的分類器
Training score : 0.9990603330957164
準確率 : 0.9947521865889213
[[3402    1]
 [  17   10]]
這是 贈品 的分類器
Training score : 0.960987622318709
準確率 : 0.8790087463556852
[[2401  235]
 [ 180  614]]
這是 跟鞋 的分類器
Training score : 0.9741753612857236
準確率 : 0.9478134110787172
[[3239   36]
 [ 143   12]]
這是 婚鞋 的分類器
Training score : 0.9919318255459789
準確率 : 0.9833819241982508
[[3372   10]
 [  47    1]]
這是 膝靴 的分類器
Training score : 0.9758278789449809
準確率 : 0.9489795918367347
[[3220   45]
 [ 130   35]]
這是 NG鞋 的分類器
Training score : 0.9991899423238935
準確率 : 0.9944606413994169
[[3410    3]
 [  16    1]]
這是 身體保養 的分類器
Training score : 0.9699630613699696
準確率 

In [55]:
print("Training Score為1的Tag數:", NumberOfScore1)
print("樣本數不足的Tag")
for i in TagDictionary.keys():
    if sum(TagDictionary[i]) <= 3:
        print(i)

Training Score為1的Tag數: 15
樣本數不足的Tag
菱格紋包
帆布鞋
黛妃包
餅乾鞋
尖頭靴
馬丁靴
方頭鞋
劍橋包
腋下包
藤編包
鍊條包
拓特包
跟靴
蛋糕鞋
豆豆鞋
艾瑪包
瑪麗珍鞋


### Season 2 (4-6月)(2014-2019)

In [56]:
result = get_season(combinedWithProduct, 2014, 2)
for i in range(2015, 2020,1):
    a = get_season(combinedWithProduct, i, 2)
    result = pd.concat([result, a])
ResultSeason2 = result.dropna()

2014-04-01 2014-07-01
2015-04-01 2015-07-01
2016-04-01 2016-07-01
2017-04-01 2017-07-01
2018-04-01 2018-07-01
2019-04-01 2019-07-01


In [57]:
ResultSeason2 = AgeCoding(ResultSeason2)
ResultSeason2["SalesLog"] = np.log(ResultSeason2["sales"])
ResultSeason2["ReviseMemberCardLevel"] = ResultSeason2["MemberCardLevel"] / 10


C:\Users\timmy\AppData\Local\Temp\ipykernel_28212\2978066741.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ResultSeason2["SalesLog"] = np.log(ResultSeason2["sales"])
C:\Users\timmy\AppData\Local\Temp\ipykernel_28212\2978066741.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ResultSeason2["ReviseMemberCardLevel"] = ResultSeason2["MemberCardLevel"] / 10


In [58]:
TagDictionary = GetTag(ResultSeason2, ProductTag)

In [59]:
NumberOfScore1 = ModelTraining(ResultSeason2, TagDictionary)
    

這是 小白鞋 的分類器
Training score : 0.8805886362725049
準確率 : 0.7940115440115441
[[1923  142]
 [ 429  278]]
這是 菱格紋包 的分類器
Training score : 1.0
準確率 : 1.0
[[2772]]
這是 鞋用配件 的分類器
Training score : 0.9910982798027186
準確率 : 0.93001443001443
[[2452   74]
 [ 120  126]]
這是 短靴 的分類器
Training score : 0.9912185733188981
準確率 : 0.9823232323232324
[[2722   12]
 [  37    1]]
這是 拖鞋 的分類器
Training score : 0.9997995108063675
準確率 : 0.9996392496392497
[[2771    0]
 [   1    0]]
這是 贈品 的分類器
Training score : 0.9796302979269418
準確率 : 0.9148629148629148
[[2082  128]
 [ 108  454]]
這是 跟鞋 的分類器
Training score : 0.9679217290188059
準確率 : 0.9411976911976911
[[2602   27]
 [ 136    7]]
這是 婚鞋 的分類器
Training score : 0.9910180841252656
準確率 : 0.9855699855699855
[[2729    6]
 [  34    3]]
這是 膝靴 的分類器
Training score : 0.9995188259352821
準確率 : 0.9989177489177489
[[2769    0]
 [   3    0]]
這是 NG鞋 的分類器
Training score : 1.0
準確率 : 1.0
[[2772]]
這是 身體保養 的分類器
Training score : 0.9941056177072056
準確率 : 0.9603174603174603
[[2635   36]
 [  74   27]]
這

In [60]:
print("Training Score為1的Tag數:", NumberOfScore1)
print("樣本數不足的Tag")
for i in TagDictionary.keys():
    if sum(TagDictionary[i]) <= 3:
        print(i)

Training Score為1的Tag數: 26
樣本數不足的Tag
菱格紋包
NG鞋
老爹鞋
帆布鞋
黛妃包
包用配件
凱莉包
餅乾鞋
尖頭靴
馬丁靴
腰包
方頭鞋
休閒包
劍橋包
相機包
NG包
腋下包
機車靴
拓特包
短鞋
跟靴
蛋糕鞋
豆豆鞋
艾瑪包
情侶鞋
瑪麗珍鞋


### Season 3 (7-9月)(2014-2019)

In [61]:
result = get_season(combinedWithProduct, 2014, 3)
for i in range(2015, 2020,1):
    a = get_season(combinedWithProduct, i, 3)
    result = pd.concat([result, a])
ResultSeason3 = result.dropna()

2014-07-01 2014-10-01
2015-07-01 2015-10-01
2016-07-01 2016-10-01
2017-07-01 2017-10-01
2018-07-01 2018-10-01
2019-07-01 2019-10-01


In [62]:
ResultSeason3 = AgeCoding(ResultSeason3)
ResultSeason3["SalesLog"] = np.log(ResultSeason3["sales"])
ResultSeason3["ReviseMemberCardLevel"] = ResultSeason3["MemberCardLevel"] / 10


C:\Users\timmy\AppData\Local\Temp\ipykernel_28212\3584889262.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ResultSeason3["SalesLog"] = np.log(ResultSeason3["sales"])
C:\Users\timmy\AppData\Local\Temp\ipykernel_28212\3584889262.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ResultSeason3["ReviseMemberCardLevel"] = ResultSeason3["MemberCardLevel"] / 10


In [63]:
TagDictionary = GetTag(ResultSeason3, ProductTag)

In [64]:
NumberOfScore1 = ModelTraining(ResultSeason3, TagDictionary)
    

這是 小白鞋 的分類器
Training score : 0.8520052212269883
準確率 : 0.7464560550830296
[[1469  241]
 [ 385  374]]
這是 菱格紋包 的分類器
Training score : 1.0
準確率 : 1.0
[[2469]]
這是 鞋用配件 的分類器
Training score : 0.9889273979385156
準確率 : 0.9254759011745646
[[2195   60]
 [ 124   90]]
這是 短靴 的分類器
Training score : 0.9867668902192015
準確率 : 0.9708383961117861
[[2396   11]
 [  61    1]]
這是 拖鞋 的分類器
Training score : 0.9998199576900572
準確率 : 0.9995949777237748
[[2468    0]
 [   1    0]]
這是 贈品 的分類器
Training score : 0.9843813296124589
準確率 : 0.9295261239368166
[[1365  121]
 [  53  930]]
這是 跟鞋 的分類器
Training score : 0.9674573524778323
準確率 : 0.9315512353179425
[[2296   29]
 [ 140    4]]
這是 婚鞋 的分類器
Training score : 0.9910879056578296
準確率 : 0.9761036857027137
[[2408    9]
 [  50    2]]
這是 膝靴 的分類器
Training score : 0.9987397038304001
準確率 : 0.9979748886188741
[[2464    0]
 [   5    0]]
這是 NG鞋 的分類器
Training score : 0.9994148624926857
準確率 : 0.9898744430943702
[[2443    9]
 [  16    1]]
這是 身體保養 的分類器
Training score : 0.9952738893640005
準確率

In [65]:
print("Training Score為1的Tag數:", NumberOfScore1)
print("樣本數不足的Tag")
for i in TagDictionary.keys():
    if sum(TagDictionary[i]) <= 3:
        print(i)

Training Score為1的Tag數: 21
樣本數不足的Tag
菱格紋包
老爹鞋
帆布鞋
黛妃包
凱莉包
餅乾鞋
尖頭靴
馬丁靴
方頭鞋
劍橋包
相機包
NG包
腋下包
拓特包
短鞋
跟靴
蛋糕鞋
豆豆鞋
艾瑪包
情侶鞋
瑪麗珍鞋


### Season 4 (10-12月)(2014-2019)

In [66]:
result = get_season(combinedWithProduct, 2013, 4)
for i in range(2014, 2020,1):
    a = get_season(combinedWithProduct, i, 4)
    result = pd.concat([result, a])
ResultSeason4 = result.dropna()

2013-10-01 2014-01-01
2014-10-01 2015-01-01
2015-10-01 2016-01-01
2016-10-01 2017-01-01
2017-10-01 2018-01-01
2018-10-01 2019-01-01
2019-10-01 2020-01-01


In [67]:
ResultSeason4 = AgeCoding(ResultSeason4)
ResultSeason4["SalesLog"] = np.log(ResultSeason4["sales"])
ResultSeason4["ReviseMemberCardLevel"] = ResultSeason4["MemberCardLevel"] / 10


C:\Users\timmy\AppData\Local\Temp\ipykernel_28212\2157600413.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ResultSeason4["SalesLog"] = np.log(ResultSeason4["sales"])
C:\Users\timmy\AppData\Local\Temp\ipykernel_28212\2157600413.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ResultSeason4["ReviseMemberCardLevel"] = ResultSeason4["MemberCardLevel"] / 10


In [68]:
TagDictionary = GetTag(ResultSeason4, ProductTag)

In [69]:
NumberOfScore1 = ModelTraining(ResultSeason4, TagDictionary)
    

這是 小白鞋 的分類器
Training score : 0.89822376155826
準確率 : 0.7938232770946526
[[2569  257]
 [ 464  207]]
這是 菱格紋包 的分類器
Training score : 1.0
準確率 : 1.0
[[3497]]
這是 鞋用配件 的分類器
Training score : 0.9861142003749483
準確率 : 0.9030597655132971
[[2989  150]
 [ 189  169]]
這是 短靴 的分類器
Training score : 0.9173842585237203
準確率 : 0.7955390334572491
[[2475  301]
 [ 414  307]]
這是 拖鞋 的分類器
Training score : 0.9995869212926186
準確率 : 0.9971404060623391
[[3487    0]
 [  10    0]]
這是 贈品 的分類器
Training score : 0.9439483969368625
準確率 : 0.8467257649413783
[[1937  353]
 [ 183 1024]]
這是 跟鞋 的分類器
Training score : 0.9698452543611579
準確率 : 0.929368029739777
[[3244   48]
 [ 199    6]]
這是 婚鞋 的分類器
Training score : 0.9901496615932128
準確率 : 0.9822705175865027
[[3435   10]
 [  52    0]]
這是 膝靴 的分類器
Training score : 0.9758507832607798
準確率 : 0.9339433800400343
[[3093  112]
 [ 119  173]]
這是 NG鞋 的分類器
Training score : 0.9997140224333514
準確率 : 0.9948527309122105
[[3478    2]
 [  16    1]]
這是 身體保養 的分類器
Training score : 0.9623780623431095
準確率 : 

In [70]:
print("Training Score為1的Tag數:", NumberOfScore1)
print("樣本數不足的Tag")
for i in TagDictionary.keys():
    if sum(TagDictionary[i]) <= 3:
        print(i)

Training Score為1的Tag數: 17
樣本數不足的Tag
菱格紋包
帆布鞋
黛妃包
凱莉包
餅乾鞋
尖頭靴
馬丁靴
方頭鞋
劍橋包
NG包
腋下包
柏金包
藤編包
鍊條包
豆豆鞋
艾瑪包
情侶鞋
瑪麗珍鞋
